# Data prepration

In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from functions import mean_gen, varx_gen, vary_gen, spherical_data, discrete_angles

2022-08-09 11:44:15.693758: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-09 11:44:15.693789: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
n = 50000


# m_th = 40
# m_ph = 40
# thetas = discrete_angles(n, m_th, 2)
# phis = discrete_angles(n, m_ph, 1)
# name = ('discrete_model_mth%.0f_mph%.0f' %(m_th, m_ph))

theta = 1
phi = 1
thetas = theta * np.ones(n)
phis = phi * np.ones(n)
name = ('discrete_model_th%.0f_ph%.0f' %(theta, phi))

x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis)

print(x_train)

[[-0.01577598  1.37980864  1.          1.        ]
 [-1.51292815  0.23023537  1.          1.        ]
 [ 0.75665344  0.38036186  1.          1.        ]
 ...
 [ 0.9220339  -0.15627483  1.          1.        ]
 [-0.31318762  0.31663098  1.          1.        ]
 [-2.00052237  0.39731297  1.          1.        ]]


# Training Interpolated Model

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]);

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

batch_size = 1000

checkpoint_path = "tutorial/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq=10*batch_size
)

log_dir = "tutorial/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS=20
model.fit(
    x_train, y_train, epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[cp_callback, tensorboard_callback]
);

model.save('3dmodels/' + name)

2022-08-09 11:47:43.170303: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-09 11:47:43.171269: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-09 11:47:43.179283: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/20
15/80 [====>.........................] - ETA: 0s - loss: 0.6581 - accuracy: 0.5993

2022-08-09 11:47:44.080150: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-09 11:47:44.080180: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-09 11:47:44.104338: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-09 11:47:44.112581: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-09 11:47:44.144457: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220809-114743/train/plugins/profile/2022_08_09_11_47_44

2022-08-09 11:47:44.150103: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tutorial/logs/20220809-114743/train/plugins/profile/2022_08_09_11_47_44/jupyter-matan.trace.json.gz
2022-08-09 11:47:44.170685: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220809-114

80/80 [==============================] - 1s 6ms/step - loss: 0.6328 - accuracy: 0.6297
Epoch 2/20
80/80 [==============================] - 0s 5ms/step - loss: 0.6229 - accuracy: 0.6416
Epoch 3/20
80/80 [==============================] - 0s 5ms/step - loss: 0.6223 - accuracy: 0.6422
Epoch 4/20
80/80 [==============================] - 0s 5ms/step - loss: 0.6218 - accuracy: 0.6416
Epoch 5/20
80/80 [==============================] - 0s 4ms/step - loss: 0.6214 - accuracy: 0.6423
Epoch 6/20
80/80 [==============================] - 0s 4ms/step - loss: 0.6216 - accuracy: 0.6423
Epoch 7/20
80/80 [==============================] - 0s 5ms/step - loss: 0.6215 - accuracy: 0.6421
Epoch 8/20
80/80 [==============================] - 0s 5ms/step - loss: 0.6212 - accuracy: 0.6424
Epoch 9/20
80/80 [==============================] - 0s 4ms/step - loss: 0.6215 - accuracy: 0.6425
Epoch 10/20
80/80 [==============================] - 0s 4ms/step - loss: 0.6210 - accuracy: 0.6426
Epoch 11/20
80/80 [===========

In [16]:
model = tf.keras.models.load_model('3dmodels/' + name)